In [2]:
import sys
import os
path = os.path.abspath('')
path = os.path.dirname(path)
sys.path.insert(1, str(path))

In [3]:
import load_csv
import pandas
from pyspark.sql import DataFrame,Window,functions as f
from pathlib import Path
df = load_csv.load("data/clean_data.csv")

your 131072x1 screen size is bogus. expect trouble
24/08/07 14:39:43 WARN Utils: Your hostname, LAPTOP-E2047RF4 resolves to a loopback address: 127.0.1.1; using 172.17.108.134 instead (on interface eth0)
24/08/07 14:39:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 14:39:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [69]:
df.show()

+----------------+-------------+---------+-----------+
|product_category|      country|unit_sold|total_sales|
+----------------+-------------+---------+-----------+
|     accessories|    Argentina|      135|   31495.98|
|     accessories|    Australia|      218|    9236.72|
|     accessories|       Bhutan|      152|   16888.05|
|     accessories|       Brazil|      267|   16406.95|
|     accessories|       Canada|      337|   20199.09|
|     accessories|         Chad|      158|    4839.46|
|     accessories|        China|      672|   69061.66|
|     accessories|      Czechia|      118|    9874.99|
|     accessories|      Finland|      160|   15631.21|
|     accessories|       France|      155|    37678.0|
|     accessories|        Gabon|      229|   21789.04|
|     accessories|Guinea-Bissau|      160|   15913.39|
|     accessories|     Honduras|      135|   10840.79|
|     accessories|        India|      108|   12777.28|
|     accessories|    Indonesia|      206|    18857.1|
|     acce

In [4]:
df: DataFrame = df.drop('order_id', 'customer_id', 'customer_name','product_id','product_name', 
        'payment_type', 'datetime', 'city', 'ecommerce_website_name',  
        'payment_txn_id','payment_txn_success')

In [5]:
df = df.withColumn("total_sales",f.round(f.col("qty") * f.col("price"),2))

In [6]:
df = df.groupBy("product_category", "country").agg(f.round(f.sum("qty"),0).cast("integer").alias("unit_sold"),
                                                   f.round(f.sum("total_sales"),2).alias("total_sales"))

In [7]:
df = df.orderBy(["product_category","country"])

In [55]:
windowSpec = Window.partitionBy("country")

In [57]:
county_max_total_sale = df.withColumn("max_total_sales", f.max(f.col("total_sales")).over(windowSpec))

In [63]:
county_max_total_sale = county_max_total_sale.filter(f.col("total_sales") == f.col("max_total_sales")).drop("max_total_sales","unit_sold")

In [61]:
county_max_unit_sold = df.withColumn("max_unit_sold", f.max(f.col("unit_sold")).over(windowSpec))

In [67]:
county_max_unit_sold = county_max_unit_sold.filter(f.col("unit_sold") == f.col("max_unit_sold")).drop("max_unit_sold","total_sales")

In [66]:
county_max_total_sale.show()

+----------------+--------------------+-----------+
|product_category|             country|total_sales|
+----------------+--------------------+-----------+
|     accessories|           Argentina|   31495.98|
|     accessories|           Australia|    9236.72|
|     accessories|              Bhutan|   16888.05|
|     accessories|              Brazil|   16406.95|
|     accessories|              Canada|   20199.09|
|     accessories|                Chad|    4839.46|
|     accessories|               China|   69061.66|
|     accessories|             Czechia|    9874.99|
|     accessories|             Finland|   15631.21|
|     accessories|              France|    37678.0|
|     accessories|               Gabon|   21789.04|
|     accessories|       Guinea-Bissau|   15913.39|
|     accessories|            Honduras|   10840.79|
|     accessories|               India|   12777.28|
|     accessories|           Indonesia|    18857.1|
|     accessories|                Iran|   49209.86|
|     access

In [68]:
county_max_unit_sold.show()

+----------------+-------------+---------+
|product_category|      country|unit_sold|
+----------------+-------------+---------+
|     accessories|    Argentina|      135|
|     accessories|    Australia|      218|
|     accessories|       Bhutan|      152|
|     accessories|       Brazil|      267|
|     accessories|       Canada|      337|
|     accessories|         Chad|      158|
|     accessories|        China|      672|
|  men's clothing|      Czechia|      156|
|     accessories|      Finland|      160|
|     accessories|       France|      155|
|     accessories|        Gabon|      229|
|     accessories|Guinea-Bissau|      160|
|     accessories|     Honduras|      135|
|     accessories|        India|      108|
|     accessories|    Indonesia|      206|
|     accessories|         Iran|      354|
|     accessories|         Iraq|      135|
|     accessories|        Japan|      286|
|     accessories|   Kazakhstan|      169|
|     accessories|        Kenya|      303|
+----------

In [8]:
category_df = df.groupBy("product_category").agg(f.round(f.sum("unit_sold"),0).cast("integer").alias("unit_sold"),f.round(f.sum("total_sales"),2).alias("total_sales"))

In [10]:
category_df = category_df.orderBy(["product_category"])

In [11]:
category_df.show()

+--------------------+---------+-----------+
|    product_category|unit_sold|total_sales|
+--------------------+---------+-----------+
|         accessories|    14844| 1924807.28|
|          appliances|     3387|  549694.33|
|      bags & luggage|      879|   28826.07|
|     beauty & health|      936|   19061.54|
|     car & motorbike|      884|   19758.06|
|grocery & gourmet...|      475|   11750.88|
|      home & kitchen|      965|   16153.74|
| industrial supplies|      645|   10168.87|
|       kids' fashion|     1640|   47143.68|
|      men's clothing|     9395|  232159.04|
|         men's shoes|     8139|  322089.42|
|               music|      232|    12435.1|
|        pet supplies|      166|     786.45|
|    sports & fitness|     1794|   64592.87|
|              stores|     3523|  184677.73|
|toys & baby products|     1105|   12470.46|
| tv, audio & cameras|     7414|  484732.99|
|    women's clothing|     9348|  194173.71|
|       women's shoes|      519|   17363.33|
+---------

In [79]:
# For python
#path = Path(__file__).parent / "category_by_country"
#df.write.csv('file://'+str(path.absolute()), header=True)

NameError: name '__file__' is not defined

In [32]:
# for jupyter
category_by_country_path = path + "/data/oluwatobi/category_by_country.csv"
df.toPandas().to_csv(str(category_by_country_path), index=False)